# Problem Statement

Company is not able to identify the applicants profile properly before loan approval and hence they are facing both business and financial loss.

# Objective

The aim is to identify patterns which indicate if a person is likely to default, which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc.

# Data Sourcing

Campany already given the dataset contains the information about past loan applicants and whether they ‘defaulted’ or not.

For this case study we need to focus only on provided dataset given by company.

In [317]:
# Importing usefull libraries to perform analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [318]:
# Read and store the dataset
# loan_ds : loan dataset
loan_ds = pd.read_csv('loan.csv', header=0, low_memory=False)

In [319]:
# Shape of loan dataset
loan_ds.shape

(39717, 111)

In [320]:
# setting to display all rows & columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Data Cleaning

Lets clean the data before doing the analysis

## Fixing Rows and Columns

Checking null values

In [330]:
loan_ds.isnull().sum().sort_values(ascending=False)

verification_status_joint         39717
annual_inc_joint                  39717
mo_sin_old_rev_tl_op              39717
mo_sin_old_il_acct                39717
bc_util                           39717
bc_open_to_buy                    39717
avg_cur_bal                       39717
acc_open_past_24mths              39717
inq_last_12m                      39717
total_cu_tl                       39717
inq_fi                            39717
total_rev_hi_lim                  39717
all_util                          39717
max_bal_bc                        39717
open_rv_24m                       39717
open_rv_12m                       39717
il_util                           39717
total_bal_il                      39717
mths_since_rcnt_il                39717
open_il_24m                       39717
open_il_12m                       39717
open_il_6m                        39717
open_acc_6m                       39717
tot_cur_bal                       39717
tot_coll_amt                      39717


By looking at the above insight:
1. From column **verification_status_joint** to **mths_since_last_major_derog** : Number of null is same as number of rows in data set. i.e. These columns having only null values. We can get rid of these columns
2. Column **mths_since_last_record** and **mths_since_last_delinq** : Very large number of null
3. Column **desc** is containing plan text. We can get rid of this as well. we can check the different values by loan_ds.desc.value_counts().

Get the total null value for desc column from above insight

In [334]:
total_null_desc = 12940
loan_ds_1 = loan_ds.loc[:, loan_ds.isnull().sum() < total_null_desc]

By looking at the head and tail seems like many columns contains lot of zeroes. Please check loan_ds_1.head(40) and loan_ds_1.tail(40)

Checking zero values

In [335]:
loan_ds_1.isin([0]).sum().sort_values(ascending=False)

delinq_amnt                   39717
acc_now_delinq                39717
tax_liens                     39678
chargeoff_within_12_mths      39661
collections_12_mths_ex_med    39661
out_prncp_inv                 38577
out_prncp                     38577
total_rec_late_fee            37671
pub_rec                       37601
pub_rec_bankruptcies          37339
collection_recovery_fee       35935
recoveries                    35499
delinq_2yrs                   35405
inq_last_6mths                19300
revol_bal                       994
dti                             183
total_pymnt_inv                 165
funded_amnt_inv                 129
total_rec_prncp                  74
last_pymnt_amnt                  74
total_rec_int                    71
total_pymnt                      16
last_pymnt_d                      0
sub_grade                         0
grade                             0
installment                       0
verification_status               0
int_rate                    

By looking at the above insight:
1. From column **delinq_amnt** to **delinq_2yrs** : Number of zero is very closer to or almost same as number of rows in data set. i.e. These columns having mostly zero values. We can get rid of these columns
2. Column **inq_last_6mths** (Inquiry in last six months) : this columns is not looking useful to analysis we perform. We can get rid of this coulmn as well.

Get the total zero value for inq_last_6mths column from above insight

In [336]:
total_zero_inq_last_6mths = 19300
loan_ds_2 = loan_ds_1.loc[:, loan_ds_1.isin([0]).sum() < total_zero_inq_last_6mths]

loan id should be unique

In [337]:
loan_ds_2.id.nunique() == loan_ds_2.shape[0]

True

Borrower member id should be unique

In [338]:
loan_ds_2.member_id.nunique() == loan_ds_2.shape[0]

True

Get rid of percentage sign and convert to float for int_rate columns

In [339]:
loan_ds_2['int_rate'] = loan_ds_2.int_rate.apply(lambda x: float(x[0:-1]))

/var/folders/7_/8px52ys552g8ncjsx6czydyh0000gn/T/ipykernel_31231/2618902194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_ds_2['int_rate'] = loan_ds_2.int_rate.apply(lambda x: float(x[0:-1]))


In [340]:
loan_ds_2.home_ownership.value_counts()

RENT        18899
MORTGAGE    17659
OWN          3058
OTHER          98
NONE            3
Name: home_ownership, dtype: int64

As per data dictonary possible values for home_ownership are RENT, OWN, MORTGAGE and OTHER. But in data set NONE is also present. Hence moving NONE to OTHER

In [341]:
loan_ds_2.home_ownership = loan_ds_2.home_ownership.apply(lambda x: 'OTHER' if x == 'NONE' else x)

/var/folders/7_/8px52ys552g8ncjsx6czydyh0000gn/T/ipykernel_31231/980230692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_ds_2.home_ownership = loan_ds_2.home_ownership.apply(lambda x: 'OTHER' if x == 'NONE' else x)


Get rid of percentage sign and convert to float for revol_util columns

In [342]:
loan_ds_2['revol_util'] = loan_ds_2.revol_util.apply(lambda x: float(str(x).rstrip('%')))

/var/folders/7_/8px52ys552g8ncjsx6czydyh0000gn/T/ipykernel_31231/3734154491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_ds_2['revol_util'] = loan_ds_2.revol_util.apply(lambda x: float(str(x).rstrip('%')))


In [343]:
loan_ds_2.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,policy_code,application_type
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Computer,860xx,AZ,27.65,Jan-85,3,13648,83.7,9,f,5863.155187,5833.84,5000.00,863.16,Jan-15,171.62,May-16,1,INDIVIDUAL
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,car,bike,309xx,GA,1.00,Apr-99,3,1687,9.4,4,f,1008.710000,1008.71,456.46,435.17,Apr-13,119.66,Sep-13,1,INDIVIDUAL
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,real estate business,606xx,IL,8.72,Nov-01,2,2956,98.5,10,f,3005.666844,3005.67,2400.00,605.67,Jun-14,649.91,May-16,1,INDIVIDUAL
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,other,personel,917xx,CA,20.00,Feb-96,10,5598,21.0,37,f,12231.890000,12231.89,10000.00,2214.92,Jan-15,357.48,Apr-16,1,INDIVIDUAL
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://lendingclub.com/browse/loanDetail.acti...,other,Personal,972xx,OR,17.94,Jan-96,15,27783,53.9,38,f,3513.330000,3513.33,2475.94,1037.39,May-16,67.79,May-16,1,INDIVIDUAL
